## 3. out-of-core learning

 In this section, we will make use of the partial_fit function of the SGDClassifier in scikit-learn to stream the documents directly from our local drive and train a logistic regression model using small minibatches of documents.

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [2]:
next(stream_docs(path='./movie_data.csv'))

('0,"Al Pacino was once an actor capable of making a role work without resorting to constant use of profanity. In other words when he could act, he didn\'t have to talk like some street junkie. McConaughey must have been impressed by Pacino because he became a promoter of the ""F"" word also. This might be the kind of society that they actually live in, but most of us have the common decency to watch what we say in mixed company. I don\'t recall the exact words that a professor used in explaining the constant use of profanity, but it was something like this. ""It shows a lack of intelligence, poor language skills and disrespect for all those that have to listen."" Maybe it is time that Al takes some acting courses again to sharpen his talent. Oh yes, the movie... Probably the worse thing that McConaughey has played in. Hopefully his next role will be in the company of more talented people. Rene Russo as always was hot."',
 0)

In [3]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [4]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

In [5]:
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

In [6]:

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)


In [7]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.868


Finally, we can use the last 5,000 documents to update our model:


In [8]:
clf = clf.partial_fit(X_test, y_test)

## Chapter 9: Embedding a Machine Learning Model into a Web Application

### Serializing fitted scikit-learn estimators

In [27]:
import pickle
import os
dest = os.path.join('movieClassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)
pickle.dump(stop, 
           open(os.path.join(dest, 'stopwords.pkl'), 'wb'),
           protocol=4)
pickle.dump(clf,
            open(os.path.join(dest, 'classifier.pkl'), 'wb'),
                protocol=4)

In [28]:
!ls movieClassifier/pkl_objects/

classifier.pkl stopwords.pkl  stopworks.pkl
